# Intial Data Preprocessing and creating a combined dataset

In [ ]:
# Importing required libraries
import pandas as pd

## Preprocess Hospital Dataset

In [20]:
# Read hospital data
hospital_data = pd.read_csv('raw/Hospital_Master_Sheet.csv')

In [21]:
# Print dataframe columns
hospital_data.columns

Index(['prvdr_num', 'prvdr_name', 'prvdr_address', 'city', 'state',
       'prov_region', 'zip', 'CBSA_CD', 'MSA_CD', 'ownership', 'urban_rural',
       'teaching', 'UCN', 'beds_grp', 'org_npi_num', 'IDN', 'IDN_parent',
       'Covid_payment'],
      dtype='object')

In [24]:
# Print dataframe
hospital_data.head()

,prvdr_num,prvdr_name,prvdr_address,city,state,prov_region,zip,CBSA_CD,MSA_CD,ownership,urban_rural,teaching,UCN,beds_grp,org_npi_num,IDN,IDN_parent,Covid_payment
0,330195,LONG ISLAND JEWISH MEDICAL CENTER,270 - 05 76TH AVENUE,NEW HYDE PARK,NY,NORTHEAST,11040,35614,35614,PRIVATE (NOT FOR PROFIT),URBAN,YES,2530.0,500+,1.285642e+09,Northwell Health (AKA: North Shore Long Island...,NaN,2.787899e+08
1,330194,MAIMONIDES MEDICAL CENTER,4802 TENTH AVENUE,BROOKLYN,NY,NORTHEAST,11219,35614,35614,PRIVATE (NOT FOR PROFIT),URBAN,YES,2537.0,500+,1.093777e+09,NaN,NaN,1.534679e+08
2,220071,MASSACHUSETTS GENERAL HOSPITAL,55 FRUIT STREET,BOSTON,MA,NORTHEAST,2114,14454,14454,PRIVATE (NOT FOR PROFIT),URBAN,YES,45256.0,500+,1.023049e+09,Partners HealthCare,NaN,1.000620e+08
3,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,NORTHEAST,7601,35614,35614,PRIVATE (NOT FOR PROFIT),URBAN,YES,2434.0,500+,1.457456e+09,Hackensack University Health Network,Hackensack Meridian Health,9.818762e+07
4,330234,WESTCHESTER MEDICAL CENTER,100 WOODS RD,VALHALLA,NY,NORTHEAST,10595,35614,35614,HOSPITAL DISTRICT OR AUTHORITY,URBAN,YES,4390.0,500+,1.932281e+09,Westchester Medical Center Health Network,Westchester Medical Center Health Network,9.588076e+07


In [28]:
# Drop irrelevant cols
hospital_data.drop(['prov_region',
                    'CBSA_CD',
                    'zip',
                    'city',
                    'state',
                    'prvdr_num',
                    'prvdr_name',
                    'prvdr_address','IDN','IDN_parent','UCN','org_npi_num'],axis=1, inplace=True)

In [29]:
# Check df
hospital_data.head()

,MSA_CD,ownership,urban_rural,teaching,beds_grp,Covid_payment
0,35614,PRIVATE (NOT FOR PROFIT),URBAN,YES,500+,2.787899e+08
1,35614,PRIVATE (NOT FOR PROFIT),URBAN,YES,500+,1.534679e+08
2,14454,PRIVATE (NOT FOR PROFIT),URBAN,YES,500+,1.000620e+08
3,35614,PRIVATE (NOT FOR PROFIT),URBAN,YES,500+,9.818762e+07
4,35614,HOSPITAL DISTRICT OR AUTHORITY,URBAN,YES,500+,9.588076e+07


### One hot encoding of categorical columns

In [30]:
# Do one hot encoding of cols
one_hot_cols = ['ownership',
                'urban_rural',
                'teaching',
                'beds_grp']

In [31]:
# Do one hot encoding
hospital_data = pd.get_dummies(hospital_data, columns=one_hot_cols, drop_first=True)

In [32]:
# Rename msa col
hospital_data = hospital_data.rename(columns={'MSA_CD':'msa'})

In [33]:
# print list of cols
hospital_data.columns

Index(['msa', 'Covid_payment', 'ownership_FEDERAL',
       'ownership_HOSPITAL DISTRICT OR AUTHORITY', 'ownership_LOCAL',
       'ownership_OTHER (SPECIFY)', 'ownership_PHYSICIAN OWNERSHIP',
       'ownership_PRIVATE (FOR PROFIT)', 'ownership_PRIVATE (NOT FOR PROFIT)',
       'ownership_STATE', 'ownership_TRIBAL', 'urban_rural_URBAN',
       'teaching_YES', 'beds_grp_100-199', 'beds_grp_200-299',
       'beds_grp_300-399', 'beds_grp_400-499', 'beds_grp_500+'],
      dtype='object')

In [34]:
df_agg_cols_mean = hospital_data.groupby('msa').agg('mean').add_suffix('_mean').reset_index()
# Agg data by count
df_agg_cols_count = hospital_data.groupby('msa').agg('count').add_suffix('_count').reset_index()
# checks if the values exists
df_agg_cols_exists = hospital_data.groupby('msa').agg('max').add_suffix('_exists').reset_index()
# Get median of cols
df_agg_cols_median = hospital_data.groupby('msa').agg('median').add_suffix('_median').reset_index()
# Get mode of cols
df_agg_cols_mode = hospital_data.groupby('msa').agg('median').add_suffix('_mode').reset_index()

In [9]:
# Merge dataframes
merged_df = pd.merge(
    df_agg_cols_mean,
    df_agg_cols_count,
    how="inner",
    on='msa')

merged_df = pd.merge(
    merged_df,
    df_agg_cols_exists,
    how="inner",
    on='msa')

merged_df = pd.merge(
    merged_df,
    df_agg_cols_median,
    how="inner",
    on='msa')

merged_df = pd.merge(
    merged_df,
    df_agg_cols_mode,
    how="inner",
    on='msa')

In [13]:
merged_df.head()

,msa,Covid_payment_mean,ownership_FEDERAL_mean,ownership_HOSPITAL DISTRICT OR AUTHORITY_mean,ownership_LOCAL_mean,ownership_OTHER (SPECIFY)_mean,ownership_PHYSICIAN OWNERSHIP_mean,ownership_PRIVATE (FOR PROFIT)_mean,ownership_PRIVATE (NOT FOR PROFIT)_mean,ownership_STATE_mean,...,ownership_PRIVATE (NOT FOR PROFIT)_mode,ownership_STATE_mode,ownership_TRIBAL_mode,urban_rural_URBAN_mode,teaching_YES_mode,beds_grp_100-199_mode,beds_grp_200-299_mode,beds_grp_300-399_mode,beds_grp_400-499_mode,beds_grp_500+_mode
0,1,4.859510e+05,0.046875,0.265625,0.078125,0.109375,0.0,0.328125,0.156250,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.000000e+00,0.153846,0.000000,0.230769,0.307692,0.0,0.000000,0.153846,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.025000e+06,0.277778,0.055556,0.055556,0.111111,0.0,0.166667,0.277778,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1.690141e+04,0.000000,0.000000,0.281690,0.056338,0.0,0.225352,0.366197,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.000000e+00,0.000000,0.357143,0.119048,0.190476,0.0,0.071429,0.142857,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
merged_df.to_csv('preprocessed/hospital_data_preprocessed.csv',index=False)

## Preprocess MSA Data

In [33]:
msa_data_df = pd.read_csv('raw/msa2020.csv', encoding='ISO-8859-1')

In [34]:
msa_data_df.head()

,CBSA,MDIV,STCOU,NAME,LSAD,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019,RESIDUAL2020
0,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,165252,165252,165590,166651,167483,...,-4,-10,23,-20,-17,-5,-5,-4,0,9
1,10180,NaN,48059.0,"Callahan County, TX",County or equivalent,13544,13545,13511,13513,13488,...,-1,-2,-2,-1,-1,0,0,0,0,-1
2,10180,NaN,48253.0,"Jones County, TX",County or equivalent,20202,20192,20237,20271,19873,...,3,14,4,2,1,-1,0,0,-2,0
3,10180,NaN,48441.0,"Taylor County, TX",County or equivalent,131506,131515,131842,132867,134122,...,-6,-22,21,-21,-17,-4,-5,-4,2,10
4,10420,NaN,NaN,"Akron, OH",Metropolitan Statistical Area,703200,703215,703056,703262,702224,...,-14,-49,-87,-121,-48,-18,-29,-11,17,19


In [35]:
# Filter MSA's
msa_data_df = msa_data_df[msa_data_df['LSAD'] == 'Metropolitan Statistical Area']

In [36]:
# Extract relevant cols
msa_data_df = msa_data_df[['CBSA', 'POPESTIMATE2020']]

In [37]:
# Rename cols
msa_data_df = msa_data_df.rename(columns={'CBSA':'msa', 'POPESTIMATE2020': 'population_estimate_2020'})

In [38]:
# View Df
msa_data_df.head()

,msa,population_estimate_2020
0,10180,173185
4,10420,701449
7,10500,145206
12,10540,131054
14,10580,878550


In [30]:
msa_data_df.to_csv('preprocessed/msa_population_2020_preprocessed.csv',index=False, encoding='utf-8')

## Preprocess Average Income Dataset

In [62]:
income_df = pd.read_csv('raw/msa_avg_income_data.csv')

In [63]:
income_df.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""00000""",United States,,MARPI,1,...,Real personal income (thousands of constant (2...,Thousands of chained 2012 dollars,13226438104,12835528512,...,13578992118,14003346000,13999660322,14549525541,15207440020,15451718336,15883838361,16323319245,16740966295,17628679929
1,"""00999""",United States (Nonmetropolitan Portion),,MARPI,1,...,Real personal income (thousands of constant (2...,Thousands of chained 2012 dollars,1759296813,1726603797,...,1793356792,1816098135,1818106548,1856006720,1901557051,1905310690,1940943207,1937183350,1974120571,2115441032
2,"""10180""","Abilene, TX (Metropolitan Statistical Area)",,MARPI,1,...,Real personal income (thousands of constant (2...,Thousands of chained 2012 dollars,6516280,6417653,...,6708568,6860195,7126927,7276677,7177769,6956803,7048707,7444927,7857445,8267776
3,"""10420""","Akron, OH (Metropolitan Statistical Area)",,MARPI,1,...,Real personal income (thousands of constant (2...,Thousands of chained 2012 dollars,31299301,29496788,...,30602854,30344944,30606195,31494725,32594998,32822702,33296567,34431634,35130005,37353892
4,"""10500""","Albany, GA (Metropolitan Statistical Area)",,MARPI,1,...,Real personal income (thousands of constant (2...,Thousands of chained 2012 dollars,5457088,5560486,...,5774623,5723514,5542406,5559543,5646138,5581975,5705634,5760768,6074927,6620352


In [64]:
# Filter the per capita income rows
income_df = income_df[income_df["Description"] == "Real per capita personal income (constant (2012) dollars) 2/"]
income_df = income_df[["GeoFIPS", "2020"]]
income_df = income_df.rename(columns={"GeoFIPS": "msa", "2020": "avg_income_2020"})

In [65]:
income_df.msa = income_df.msa.apply(lambda x: x.replace('"', ""))
income_df.msa = pd.to_numeric(income_df.msa)

In [69]:
income_df.to_csv('preprocessed/income_data_preprocessed.csv',index=False)

## Merge all preprocessed datasets to master dataset

In [97]:
# Load master dataset
priv_mcare_df = pd.read_csv('raw/priv_mcare_f_pay.csv')

# Load preprocessed hospital data
hospital_data_df = pd.read_csv('preprocessed/hospital_data_preprocessed.csv')

# Load income preprocessed data
income_data_df = pd.read_csv('preprocessed/income_data_preprocessed.csv')

# Load MSA population preprocessed data
msa_population_df = pd.read_csv('preprocessed/msa_population_2020_preprocessed.csv')

In [98]:
priv_mcare_df.head()

,msa,year,site,group,priv_count,priv_pay_mean,priv_pay_median,priv_pay_iqr,mcare_pay_mean,mcare_pay_median,mcare_pay_sd,CBSA_NAME,State,FIPS State Code,lon,lat
0,10180,2018,Inpatient,breast reconstruction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Abilene, TX",Texas,48,-99.733144,32.448736
1,10420,2018,Inpatient,breast reconstruction,8.0,19937.08375,16147.330,5692.86,NaN,NaN,NaN,"Akron, OH",Ohio,39,-81.519005,41.081445
2,10500,2018,Inpatient,breast reconstruction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Albany, GA",Georgia,13,-84.155741,31.578507
3,10540,2018,Inpatient,breast reconstruction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Albany-Lebanon, OR",Oregon,41,-122.907034,44.536512
4,10580,2018,Inpatient,breast reconstruction,4.0,14837.26000,10420.675,4474.06,NaN,NaN,NaN,"Albany-Schenectady-Troy, NY",New York,36,-73.653621,42.763648


In [99]:
hospital_data_df.head()

,msa,Covid_payment_mean,ownership_FEDERAL_mean,ownership_HOSPITAL DISTRICT OR AUTHORITY_mean,ownership_LOCAL_mean,ownership_OTHER (SPECIFY)_mean,ownership_PHYSICIAN OWNERSHIP_mean,ownership_PRIVATE (FOR PROFIT)_mean,ownership_PRIVATE (NOT FOR PROFIT)_mean,ownership_STATE_mean,...,ownership_PRIVATE (NOT FOR PROFIT)_mode,ownership_STATE_mode,ownership_TRIBAL_mode,urban_rural_URBAN_mode,teaching_YES_mode,beds_grp_100-199_mode,beds_grp_200-299_mode,beds_grp_300-399_mode,beds_grp_400-499_mode,beds_grp_500+_mode
0,1,4.859510e+05,0.046875,0.265625,0.078125,0.109375,0.0,0.328125,0.156250,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.000000e+00,0.153846,0.000000,0.230769,0.307692,0.0,0.000000,0.153846,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.025000e+06,0.277778,0.055556,0.055556,0.111111,0.0,0.166667,0.277778,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1.690141e+04,0.000000,0.000000,0.281690,0.056338,0.0,0.225352,0.366197,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.000000e+00,0.000000,0.357143,0.119048,0.190476,0.0,0.071429,0.142857,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
income_data_df.head()

,msa,avg_income_2020
0,0,8.814367e+09
1,999,1.057744e+09
2,10180,4.157758e+06
3,10420,1.870357e+07
4,10500,3.332972e+06


In [101]:
msa_population_df.head()

,msa,population_estimate_2020
0,10180,173185
1,10420,701449
2,10500,145206
3,10540,131054
4,10580,878550


In [102]:
# Merge priv_mcare_df, hospital_data_df
merged_df = pd.merge(priv_mcare_df,hospital_data_df, how='left', left_on='msa',right_on='msa')
print(merged_df.shape)

# Merge existing merged_df, income_data_df
merged_df = pd.merge(merged_df,income_data_df, how='left', left_on='msa',right_on='msa')
print(merged_df.shape)

# Merge existing merged_df, msa_population_df
merged_df = pd.merge(merged_df,msa_population_df, how='left', left_on='msa',right_on='msa')
print(merged_df.shape)

(133089, 101)
(133089, 102)
(133089, 103)


In [104]:
merged_df.head()

,msa,year,site,group,priv_count,priv_pay_mean,priv_pay_median,priv_pay_iqr,mcare_pay_mean,mcare_pay_median,...,ownership_TRIBAL_mode,urban_rural_URBAN_mode,teaching_YES_mode,beds_grp_100-199_mode,beds_grp_200-299_mode,beds_grp_300-399_mode,beds_grp_400-499_mode,beds_grp_500+_mode,avg_income_2020,population_estimate_2020
0,10180,2018,Inpatient,breast reconstruction,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4157758.0,173185.0
1,10420,2018,Inpatient,breast reconstruction,8.0,19937.08375,16147.330,5692.86,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,18703572.0,701449.0
2,10500,2018,Inpatient,breast reconstruction,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3332972.5,145206.0
3,10540,2018,Inpatient,breast reconstruction,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,2816254.5,131054.0
4,10580,2018,Inpatient,breast reconstruction,4.0,14837.26000,10420.675,4474.06,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25771091.5,878550.0


In [105]:
# Write final merged df
merged_df.to_csv('merged_dataset.csv',index=False)